#### Importing libraries

In [1]:
import pickle
import requests
import pandas as pd
from pandas_profiling import ProfileReport
import missingno as msno
import json
from tqdm.notebook import tqdm, trange
#pip install googletrans==4.0.0-rc1
import googletrans
import numpy as np
from numpy import array
from numpy import hstack
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium as fo
from neuralprophet import NeuralProphet
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, Flatten, ConvLSTM2D, Bidirectional, TimeDistributed, RepeatVector, MaxPooling1D, LSTM
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from patsy import dmatrices
import statsmodels.api as sm
from fitter import Fitter, get_common_distributions, get_distributions

%matplotlib inline


translator = googletrans.Translator()

stations = requests.get('https://opendata-download-metobs.smhi.se/api/version/latest/parameter/1/station-set/all/period/latest-hour/data.json').text
stations = json.loads(stations)
stations = pd.json_normalize(stations['station'])

parameters = requests.get('https://opendata-download-metobs.smhi.se/api/version/latest.json').text
parameters = json.loads(parameters)
parameters = pd.json_normalize(parameters['resource'])

#### ETL functions

In [2]:
def get_data(param, site_id):
    url_data = f'https://opendata-download-metobs.smhi.se/api/version/latest/parameter/{param}/station/{site_id}/period/corrected-archive/data.csv' #latest-months
    r = requests.get(url_data)
    if r.status_code == 200:
        text = r.text
        skiprows = 0
        iteration = text.splitlines()
        for line in iteration:
            if 'Datum' in line and 'Kvalitet' in line:
                skiprows = iteration.index(line)
                break
        cols = eval(str(iteration[skiprows].split(';')).split(']')[0]+']')
        if '' in cols:
            cols.remove('')
        df = pd.read_csv(url_data, sep=';', skiprows=35, index_col=False, names=cols) #int(len(iteration)*0.8) low_memory=False,
        #df.dropna(axis=1, how='all', inplace=True)
        for col in df.columns:
            if 'historika data' in col or 'Tidsutsnitt' in col:
                del df[col]
        df['station'] = site_id
        if 'Till Datum Tid (UTC)' in df.columns and "Från Datum Tid (UTC)" in df.columns:
            del df['Från Datum Tid (UTC)']
            del df['Till Datum Tid (UTC)']
            df.rename(columns={'Representativt dygn': 'Datum'}, inplace=True)
        if 'Tid (UTC)' in df.columns:
            del df['Tid (UTC)']

    return df.drop_duplicates(subset='Datum', keep='last')

def prepare_data(station):

    snow_depth = get_data('8', station)
    air_temp = get_data('2', station)

    air_temp_min = get_data('26', station) #
    air_temp_max = get_data('27', station) #
    
    precip = pd.concat([get_data('18', station), get_data('17', station)])
    precip_val = get_data('5', station)

    #
    land = get_data('40', station)
    #total_cloud = get_data('16', station)[['Datum', 'Total molnmängd']].groupby('Datum').mean()
    #

    data = snow_depth.merge(air_temp[['Datum', 'Lufttemperatur']], on=['Datum'], how='outer')

    data = data.merge(air_temp_min[['Datum', 'Lufttemperatur']], on=['Datum'], how='outer') #
    data = data.merge(air_temp_max[['Datum', 'Lufttemperatur']], on=['Datum'], how='outer') #

    data = data.merge(precip[['Datum', 'Nederbörd']], on=['Datum'], how='outer')
    data = data.merge(precip_val[['Datum', 'Nederbördsmängd']], on=['Datum'], how='outer')

    #
    data = data.merge(land[['Datum', 'Markens tillstånd']], on=['Datum'], how='outer')
    #data = data.merge(total_cloud[['lower_cloud', 'Total molnmängd']], on=['Datum'], how='outer')
    #

    del data['Kvalitet']

    return data

def get_full_data():
    list_data = []
    for key in stations['key']:
        try:
            list_data.append(prepare_data(key))
        except:
            pass

    data = pd.concat(list_data)

    #data['Nederbörd'] = data['Nederbörd'].apply(lambda x: translator.translate(x, dest='en', src='sv').text)

    data.columns = [translator.translate(x, dest='en', src='sv').text for x in data.columns]
    data = data.merge(stations[['key', 'name', 'height', 'latitude', 'longitude']], left_on = 'station', right_on = 'key', how = 'inner')

    del data['key']

    new_cols = []
    for col in data.columns:
        col = col.lower()
        col = col.replace(' ', '_')
        col = col.replace('_x', '')
        col = col.replace('_x', '')
        col = col.replace('_y', '')
        col = col.replace('.', '')
        new_cols.append(col)
    data.columns = new_cols

    return data

def pred_bin(data, features, model):

    data_pred_bin = data.sort_values(by = 'date', ascending = True).copy()

    X_normalized = data_pred_bin[features].values
    X_normalized = StandardScaler().fit_transform(X_normalized)
    df_pred = pd.DataFrame(X_normalized, columns = features)
    df_pred['date'] = data_pred_bin['date'].tolist()

    to_pred = df_pred[features]

    to_pred.insert(loc=0, column='Intercept', value=1.0)

    prediction = model.get_prediction(to_pred.values)
    print(to_pred.shape)

    return prediction

def rectify_data(data, key, from_date):
    rectified = {}
    for station in data[key].unique():
        df = data.loc[(data[key] == station) & (pd.to_datetime(data['date']) > pd.to_datetime(from_date))].copy(deep=True)
        df['snow_depth'].fillna(value=0.0, inplace=True)
        #df.interpolate(method='linear', inplace=True)
        #if df.isnull().any().any() == False:
        rectified[station] = df



    rectified = pd.concat([rectified[x] for x in rectified.keys()])

    return rectified

#### Model functions

In [3]:
def cnn(data, features, target, filter_size, epochs):
    
    scaler = StandardScaler()
    x =  scaler.fit_transform(data[features].values)
    x = x.reshape(x.shape[0], x.shape[1], 1)
    y = data[target].values

    xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.1, shuffle = False)

    early_stopping = EarlyStopping(monitor='val_loss', patience = 5)

    model = Sequential()
    model.add(Conv1D(32, filter_size, activation="relu", input_shape=(len(features), 1)))
    model.add(Flatten())
    model.add(Dense(2, activation="relu"))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")

    history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), batch_size=64, epochs=epochs, verbose=0, callbacks = [early_stopping])
    # plot history
    #plt.plot(history.history['loss'], label='train')
    #plt.plot(history.history['val_loss'], label='test')

    ypred = model.predict(xtest)
    rmse = np.sqrt(mean_squared_error(ytest, ypred))

    #print(model.evaluate(xtrain, ytrain))
    #print("RMSE: %.4f" % np.sqrt(mean_squared_error(ytest, ypred)))
    #plt.figure(figsize=(25, 10))
    #plt.plot(ytest, lw=0.8, color="blue", label="original")
    #plt.plot(ypred, lw=0.8, color="red", label="predicted")
    #plt.legend()
    #plt.show()

    return model, rmse, scaler

def pred_cnn(data, date_col, features, target, model, scaler, evaluate):

    x = data[features].values
    x =  scaler.transform(x)

    x = x.reshape(x.shape[0], x.shape[1], 1)

    predictions = model.predict(x).flatten()

    rmse = None
    fig = None
    
    if evaluate == True:
        actuals = data[target].values
        rmse = np.sqrt(mean_squared_error(actuals, predictions))

        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=actuals, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=predictions, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'CNN',
                title_x=0.5
            )

    return predictions, rmse, fig

In [4]:
def multi_lstm(data, features, target, n_steps, epochs):
    
    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
        
    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array(data[target])
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # choose a number of time steps
    n_steps = n_steps

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.1, shuffle = False)

    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    early_stopping = EarlyStopping(monitor='loss', patience = 2)

    ## fit the model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(xtrain, ytrain, epochs=epochs, verbose=0, callbacks=[early_stopping])
    #plt.plot(history.history['loss'])

    yhat = model.predict(xtest).flatten()
    rmse = np.sqrt(mean_squared_error(y_true = ytest, y_pred = yhat))


    return model, rmse

def pred_multi_lstm(data, date_col, features, target, n_steps, model, evaluate):

    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array([np.nan for i in range(arr.shape[0])]) if evaluate == False else data[target].values
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)

    # Make predictions
    yhat = model.predict(X).flatten()

    # RMSE
    rmse = None
    fig = None
    if evaluate == True:
        y_true = data[target].tolist()[n_steps-1:]

        rmse = np.sqrt(mean_squared_error(y_true = y_true, y_pred = yhat))
        
        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=y_true, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=yhat, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'Multivariate LSTM',
                title_x=0.5
            )

    return yhat, rmse, fig

In [5]:
def cnn_lstm(data, features, target, n_steps, epochs):
    
    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
        
    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array(data[target])
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # choose a number of time steps
    n_steps = n_steps

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.1, shuffle = False)

    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    early_stopping = EarlyStopping(monitor='loss', patience = 2)

    # define number of subsequences and reshape for cnn-lstm
    n_seq = 1
    xtrain = xtrain.reshape((xtrain.shape[0], n_seq, n_steps, n_features))
    xtest = xtest.reshape((xtest.shape[0], n_seq, n_steps, n_features))
    
    # fit the model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(xtrain, ytrain, epochs=epochs, verbose=0, callbacks=[early_stopping])
    #plt.plot(history.history['loss'])

    yhat = model.predict(xtest).flatten()
    rmse = np.sqrt(mean_squared_error(y_true = ytest, y_pred = yhat))


    return model, rmse

def pred_cnn_lstm(data, date_col, features, target, n_steps, model, evaluate):

    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array([np.nan for i in range(arr.shape[0])]) if evaluate == False else data[target].values
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    n_seq = 1
    n_features = X.shape[2]
    X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

    # Make predictions
    yhat = model.predict(X).flatten()

    # RMSE
    rmse = None
    fig = None
    if evaluate == True:
        y_true = data[target].tolist()[n_steps-1:]

        rmse = np.sqrt(mean_squared_error(y_true = y_true, y_pred = yhat))
        
        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=y_true, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=yhat, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'CNN-LSTM',
                title_x=0.5
            )

    return yhat, rmse, fig

In [6]:
def conv_lstm(data, features, target, n_steps, epochs):

    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
        
    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array(data[target])
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # choose a number of time steps
    n_steps = n_steps

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.1, shuffle = False)

    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    early_stopping = EarlyStopping(monitor='loss', patience = 2)

    # define number of subsequences and reshape for cnn-lstm
    n_seq = 1
    xtrain = xtrain.reshape((xtrain.shape[0], n_seq, 1, n_steps, n_features))
    xtest = xtest.reshape((xtest.shape[0], n_seq, 1, n_steps, n_features))
    
    # fit the model
    
    model = Sequential()
    model.add(ConvLSTM2D(filters=n_features, kernel_size=(1,6), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(Flatten())
    model.add(Dense(n_features*4))
    model.add(Dense(2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(xtrain, ytrain, epochs=epochs, verbose=0, callbacks=[early_stopping])
    #plt.plot(history.history['loss'])

    yhat = model.predict(xtest).flatten()
    rmse = np.sqrt(mean_squared_error(y_true = ytest, y_pred = yhat))

    return model, rmse

def pred_conv_lstm(data, date_col, features, target, n_steps, model, evaluate):

    # split a multivariate sequence into samples
    def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

    # define input sequence and convert to [rows, columns] structure
    data_list = []
    for i in features:
        arr = array(data[i])
        arr = arr.reshape((len(arr), 1))
        data_list.append(arr)
    out_seq = array([np.nan for i in range(arr.shape[0])]) if evaluate == False else data[target].values
    out_seq = out_seq.reshape((len(out_seq), 1))
    data_list.append(out_seq)

    # horizontally stack columns
    dataset = hstack([i for i in data_list])

    # convert into input/output
    X, y = split_sequences(dataset, n_steps)
    n_seq = 1
    n_features = X.shape[2]
    X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

    # Make predictions
    yhat = model.predict(X).flatten()

    # RMSE
    rmse = None
    fig = None
    if evaluate == True:
        y_true = data[target].tolist()[n_steps-1:]

        rmse = np.sqrt(mean_squared_error(y_true = y_true, y_pred = yhat))
        
        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=y_true, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=yhat, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'CNN-LSTM',
                title_x=0.5
            )

    return yhat, rmse, fig

In [7]:
def lin_reg(data, features, target):

    X_normalized = data[features].values
    scaler = StandardScaler().fit(X_normalized)
    X_normalized = scaler.transform(X_normalized)
    y_normalized = data[target].values

    X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_normalized, y_normalized, test_size = 0.1, random_state = 324, shuffle = False)

    regressor_n = LinearRegression()
    regressor_n.fit(X_train_n, y_train_n)

    y_prediction_n = regressor_n.predict(X_test_n)

    #for i in range(len(y_prediction_n)):
    #    if y_prediction_n[i]<0:
    #        y_prediction_n[i] = 0

    RMSE_n = np.sqrt(mean_squared_error(y_true = y_test_n, y_pred = y_prediction_n))

    return regressor_n, RMSE_n, scaler

def pred_lin_reg(data, date_col, features, target, model, scaler, evaluate):
    
    X = data[features].values
    X = scaler.transform(X)

    predictions = model.predict(X)

    rmse_st = None
    fig = None

    if evaluate == True:
        y_true = data[target].tolist()

        rmse_st = np.sqrt(mean_squared_error(y_true = y_true, y_pred = predictions))
        
        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=y_true, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=predictions, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'Linear Regression',
                title_x=0.5
            )
    
    return predictions, rmse_st, fig

In [8]:
def binomial_glm(data_input, features, target, date_col):
    
    data = data_input.sort_values(by = date_col, ascending = True).copy()

    X_normalized = data[features].values
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X_normalized)
    y_normalized = data[target].values

    df = pd.DataFrame(X_normalized, columns = features)
    df['date'] = data['date'].tolist()
    df[target] = y_normalized

    new_cols = []
    for col in df.columns:
        col = col.replace('.', '')
        col = col.replace(' ', '_')
        new_cols.append(col)

    new_feat = []
    for feat in features:
        feat = feat.replace('.', '')
        feat = feat.replace(' ', '_')
        new_feat.append(feat)

    df.columns = new_cols

    target = target.replace('.', '')
    target = target.replace(' ', '_')

    expr = f"""{target} ~ """
    for feature in new_feat:
        expr = expr + f"{feature} + "
    expr = expr[:-3]

    mask = np.random.rand(len(df)) < 0.9
    df_train = df[mask]
    df_test = df[~mask]

    y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
    y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

    model = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()

    #X_train = sm.add_constant(X_train)
    #X_test = sm.add_constant(X_test)

    predictions = model.get_prediction(X_test)
    predictions_summary_frame = predictions.summary_frame()

    predicted_counts = predictions_summary_frame['mean']
    actual_counts = y_test['snow_depth']
    #fig = plt.figure(figsize=(25,10))
    #fig.suptitle('Predicted versus actual')
    #plt.plot(X_test.index, predicted_counts, label = 'Predicted')
    #plt.plot(X_test.index, actual_counts, label = 'Actual')
    #plt.show()

    RMSE_n = np.sqrt(mean_squared_error(y_true = actual_counts, y_pred = predicted_counts))

    return model, RMSE_n, scaler

def pred_bin(data, date_col, features, target, model, scaler, evaluate):

    data_pred_bin = data.sort_values(by = date_col, ascending = True).copy()

    X_normalized = data_pred_bin[features].values
    X_normalized = scaler.transform(X_normalized)
    df_pred = pd.DataFrame(X_normalized, columns = features)
    df_pred[date_col] = data_pred_bin[date_col].tolist()

    to_pred = df_pred[features]

    to_pred.insert(loc=0, column='Intercept', value=1.0)

    predictions = model.get_prediction(to_pred.values).summary_frame()['mean']
    rmse = None
    fig = None

    if evaluate == True:
        y_true = data[target].tolist()

        rmse = np.sqrt(mean_squared_error(y_true = y_true, y_pred = predictions))
        
        fig = make_subplots()

        fig.add_trace(
                go.Scatter(x=data[date_col], y=y_true, 
                    name="Actuals",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.add_trace(
                go.Scatter(x=data[date_col], y=predictions, 
                    name="Predictions",
                    mode = 'lines'
                ),
            row=1,
            col=1
        )

        fig.update_layout(
                title = 'Binomial GLM',
                title_x=0.5
            )

    return predictions, rmse, fig

#### Ingesting data

In [10]:
full_data = get_full_data()

full_data.columns = ['date', 'snow_depth', 'station', 'air_temperature_mean', 'air_temperature_min',
       'air_temperature_max', 'precipitation', 'rainfall', 'the_state_of_the_land', 'name', 'height', 'latitude', 'longitude']

#### Cleaning data

In [11]:
rectified = rectify_data(full_data, 'station', '1992-12-31')
#backup = rectified.dropna().reset_index(drop=True)

In [ ]:
data = {}

for station in tqdm(rectified['station'].unique(), desc='Station'):

    df = rectified.loc[rectified['station'] == station].reset_index(drop=True)

    df['date'] = pd.to_datetime(df['date'])
    start, end = df['date'].min(), df['date'].max()
    all_timepoints = pd.date_range(start, end, freq='D').to_series(name='date')
    df = df.merge(all_timepoints , on='date', how='outer', sort=True)

    df['year'] = pd.to_datetime(df['date']).dt.year
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['day'] = pd.to_datetime(df['date']).dt.day

    df['snow_depth'].fillna(value=0.0, inplace=True)

    for col in ['name', 'height', 'latitude', 'longitude', 'station']:
        for i in df[col].unique():
            if pd.isnull(i) == False:
                df[col].fillna(value=i, inplace=True)

    for col in df.columns:
        if df[col].isna().any() == True:
            for row in df.index:
                if pd.isnull(df.loc[row, col]) == True:
                    day = df.loc[row, 'day']
                    month = df.loc[row, 'month']
                    if col not in ['precipitation', 'the_state_of_the_land']:
                        s = df.loc[(df['day'] == day) & (df['month'] == month) & (df['station'] == station)][col].mean()
                        df.loc[row, col] = s
                    else:
                        s = df.loc[(df['day'] == day) & (df['month'] == month) & (df['station'] == station)][col].mode()
                        try:
                            s = s.iloc[0]
                            df.loc[row, col] = s
                        except:
                            continue
    

    if df.isna().any().any() == False:
        data[station] = df

total_data = pd.concat([data[x] for x in data.keys()])

In [9]:
total_data = pd.read_csv('backup.csv')
del total_data['Unnamed: 0']

#total_data = data
data = total_data.loc[total_data['year'] >= 1993].reset_index(drop=True)

In [10]:
precip_encoder = LabelEncoder()
data['precipitation'] = precip_encoder.fit_transform(data['precipitation'])

#### Standardize precipitation encoding to one station and select features

In [11]:
list_dict = data.loc[data['station'] == 105370, 'precipitation'].unique().tolist()
for i in data.index:
    if data.loc[i, 'precipitation'] not in list_dict:
        data.loc[i, 'precipitation'] = np.nan

data['precipitation'].ffill(inplace=True)

In [12]:
features = ['air_temperature_min', 'precipitation', 'rainfall', 'month']
target = 'snow_depth'
date_col = 'date'

### Linear Regression

##### Single Station Training

In [13]:
station_selected = 105370
single_station_train = data.loc[data['station'] == station_selected].reset_index(drop=True)
lin_reg_single_model, lin_reg_single_rmse, lin_reg_single_scaler = lin_reg(single_station_train, features, target)

rmses_single_lin_reg = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_lin_reg(station_data, date_col, features, target, lin_reg_single_model, lin_reg_single_scaler, True)
    rmses_single_lin_reg.append((rmse, station, lat, lon, color))

rmses_single_lin_reg.sort(key = lambda x: x[1])

In [14]:
linear_reg_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_lin_reg], 
    y=[x[0] for x in rmses_single_lin_reg],
    color=[x[4] for x in rmses_single_lin_reg],
    title='RMSE - Linear Regression Single Station'
    )

linear_reg_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

linear_reg_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_lin_reg]))

linear_reg_rmse_single.show()

##### Multiple Station Training

In [15]:
multi_station_train = data.sample(n=250000, weights='station', random_state=1)
lin_reg_multi_model, lin_reg_multi_rmse, lin_reg_multi_scaler = lin_reg(multi_station_train, features, target)

rmses_multi_lin_reg = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    station_data = station_data[~station_data.index.isin(multi_station_train.index.tolist())]

    predictions, rmse, fig = pred_lin_reg(station_data, date_col, features, target, lin_reg_multi_model, lin_reg_multi_scaler, True)
    rmses_multi_lin_reg.append((rmse, station, lat, lon))

rmses_multi_lin_reg.sort(key = lambda x: x[0])

In [16]:
linear_reg_rmse_multi = px.bar( 
    x=[str(x[1]) for x in rmses_multi_lin_reg], 
    y=[x[0] for x in rmses_multi_lin_reg],
    title='RMSE - Linear Regression Multi Station'
    )

linear_reg_rmse_multi.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

linear_reg_rmse_multi.add_hline(y = np.mean([x[0] for x in rmses_multi_lin_reg]))

linear_reg_rmse_multi.show()

##### Individual Station Training

In [17]:
rmses_indiv_lin_reg = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    lin_reg_indiv_model, lin_reg_indiv_rmse, lin_reg_indiv_scaler = lin_reg(station_data, features, target)

    rmses_indiv_lin_reg.append((lin_reg_indiv_rmse, station, lat, lon))

rmses_indiv_lin_reg.sort(key = lambda x: x[0])

In [18]:
linear_reg_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_lin_reg], 
    y=[x[0] for x in rmses_indiv_lin_reg],
    title='RMSE - Linear Regression Individual Station'
    )

linear_reg_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

linear_reg_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_lin_reg]))

linear_reg_rmse_indiv.show()

##### Regression RMSE Map

In [19]:
lat_init = data.loc[data['station'] == station_selected]['latitude'].values[0]
lon_init = data.loc[data['station'] == station_selected]['longitude'].values[0]

sp = fo.Map(location=[lat_init, lon_init], zoom_start=8)

for i in rmses_indiv_lin_reg:
    fo.CircleMarker(
        [i[2], i[3]],
        radius=50*i[0],
        color='blue',
        fill_color='blue',
        fill=True,
        fill_opacity=0.7
        ).add_to(sp)
sp

##### Regression Viz

In [20]:
final_lin_predictions, final_lin_rmse, final_lin_fig = pred_lin_reg(single_station_train, date_col, features, target, lin_reg_single_model, lin_reg_single_scaler, True)

final_lin_fig.add_hline(
    y = final_lin_rmse, 
    annotation_text = str(round(final_lin_rmse, 3)),
    annotation=dict(font_size=20, font_family="Times New Roman")
)

final_lin_fig

### Binomial GLM

#### Single Station Training

In [21]:
bin_glm_single_model, bin_glm_single_rmse, bin_glm_single_scaler = binomial_glm(single_station_train, features, target, date_col)

rmses_single_bin_glm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_bin(station_data, date_col, features, target, bin_glm_single_model, bin_glm_single_scaler, True)
    rmses_single_bin_glm.append((rmse, station, lat, lon, color))

rmses_single_bin_glm.sort(key = lambda x: x[0])

In [22]:
bin_glm_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_bin_glm], 
    y=[x[0] for x in rmses_single_bin_glm],
    color=[x[4] for x in rmses_single_bin_glm],
    title='RMSE - Binomial GLM Single Station'
    )

bin_glm_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

bin_glm_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_bin_glm]))

bin_glm_rmse_single.show()

#### Multiple Station Training

In [23]:
bin_glm_multi_model, bin_glm_multi_rmse, bin_glm_multi_scaler = binomial_glm(multi_station_train, features, target, date_col)

rmses_multi_bin_glm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    station_data = station_data[~station_data.index.isin(multi_station_train.index.tolist())]

    predictions, rmse, fig = pred_bin(station_data, date_col, features, target, bin_glm_single_model, bin_glm_single_scaler, True)
    rmses_multi_bin_glm.append((rmse, station, lat, lon))

rmses_multi_bin_glm.sort(key = lambda x: x[0])

In [24]:
bin_glm_rmse_multi = px.bar( 
    x=[str(x[1]) for x in rmses_multi_bin_glm], 
    y=[x[0] for x in rmses_multi_bin_glm],
    title='RMSE - Binomial GLM Multi Station'
    )

bin_glm_rmse_multi.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

bin_glm_rmse_multi.add_hline(y = np.mean([x[0] for x in rmses_multi_bin_glm]))

bin_glm_rmse_multi.show()

#### Individual Station Training

In [25]:
rmses_indiv_bin_glm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    bin_glm_indiv_model, bin_glm_indiv_rmse, bin_glm_indiv_scaler = binomial_glm(station_data, features, target, date_col)

    rmses_indiv_bin_glm.append((bin_glm_indiv_rmse, station, lat, lon))

rmses_indiv_bin_glm.sort(key = lambda x: x[0])

In [26]:
bin_glm_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_bin_glm], 
    y=[x[0] for x in rmses_indiv_bin_glm],
    title='RMSE - Binomial GLM Individual Station'
    )

bin_glm_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

bin_glm_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_lin_reg]))

bin_glm_rmse_indiv.show()

### CNN

#### Single Station Training

In [27]:
cnn_single_model, cnn_single_rmse, cnn_single_scaler = cnn(single_station_train, features, target, 3, 100)

rmses_single_cnn = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_cnn(station_data, date_col, features, target, cnn_single_model, cnn_single_scaler, True)
    rmses_single_cnn.append((rmse, station, lat, lon, color))

rmses_single_cnn.sort(key = lambda x: x[0])

In [28]:
cnn_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_cnn], 
    y=[x[0] for x in rmses_single_cnn],
    color=[x[4] for x in rmses_single_cnn],
    title='RMSE - CNN Single Station'
    )

cnn_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

cnn_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_cnn]))

cnn_rmse_single.show()

#### Multiple Station Training

In [29]:
cnn_multi_model, cnn_multi_rmse, cnn_multi_scaler = cnn(multi_station_train, features, target, 3, 100)

rmses_multi_cnn = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    station_data = station_data[~station_data.index.isin(multi_station_train.index.tolist())]

    predictions, rmse, fig = pred_cnn(station_data, date_col, features, target, cnn_single_model, cnn_single_scaler, True)

    rmses_multi_cnn.append((rmse, station, lat, lon))

rmses_multi_cnn.sort(key = lambda x: x[0])

In [30]:
cnn_rmse_multi = px.bar( 
    x=[str(x[1]) for x in rmses_multi_cnn], 
    y=[x[0] for x in rmses_multi_cnn],
    title='RMSE - CNN Multi Station'
    )

cnn_rmse_multi.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

cnn_rmse_multi.add_hline(y = np.mean([x[0] for x in rmses_multi_cnn]))

cnn_rmse_multi.show()

#### Individual Station Training

In [31]:
rmses_indiv_cnn = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    cnn_indiv_model, cnn_indiv_rmse, cnn_indiv_scaler = cnn(station_data, features, target, 3, 100)

    rmses_indiv_cnn.append((cnn_indiv_rmse, station, lat, lon))

rmses_indiv_cnn.sort(key = lambda x: x[0])

In [32]:
cnn_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_cnn], 
    y=[x[0] for x in rmses_indiv_cnn],
    title='RMSE - CNN Individual Station'
    )

cnn_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

cnn_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_lin_reg]))

cnn_rmse_indiv.show()

### Multi LSTM

#### Single Station Training

In [33]:
n_steps = 28
epochs = 15

multi_lstm_single_model, multi_lstm_single_model_rmse = multi_lstm(single_station_train, features, target, n_steps, epochs)

rmses_single_multi_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_multi_lstm(station_data, date_col, features, target, n_steps, multi_lstm_single_model, True)
    rmses_single_multi_lstm.append((rmse, station, lat, lon, color))

rmses_single_multi_lstm.sort(key = lambda x: x[0])

In [34]:
multi_lstm_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_multi_lstm],
    y=[x[0] for x in rmses_single_multi_lstm],
    #color=[x[4] for x in rmses_single_multi_lstm],
    title='RMSE - Multi LSTM Single Station'
    )

multi_lstm_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

multi_lstm_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_multi_lstm]))

multi_lstm_rmse_single.show()

#### Individual Station Training

In [35]:
rmses_indiv_multi_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    multi_lstm_indiv_model, multi_lstm_indiv_rmse = multi_lstm(station_data, features, target, n_steps, epochs)

    rmses_indiv_multi_lstm.append((multi_lstm_indiv_rmse, station, lat, lon))

rmses_indiv_multi_lstm.sort(key = lambda x: x[0])

In [36]:
multi_lstm_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_multi_lstm],
    y=[x[0] for x in rmses_indiv_multi_lstm],
    #color=[x[4] for x in rmses_single_multi_lstm],
    title='RMSE - Multi LSTM Individual Station'
    )

multi_lstm_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

multi_lstm_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_multi_lstm]))

multi_lstm_rmse_indiv.show()

### CNN LSTM

#### Single Station Training

In [37]:
n_steps = 28
epochs = 15

cnn_lstm_single_model, cnn_lstm_single_rmse = cnn_lstm(single_station_train, features, target, n_steps, epochs)

rmses_single_cnn_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_cnn_lstm(station_data, date_col, features, target, n_steps, cnn_lstm_single_model, True)
    rmses_single_cnn_lstm.append((rmse, station, lat, lon, color))

rmses_single_cnn_lstm.sort(key = lambda x: x[0])

In [38]:
cnn_lstm_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_cnn_lstm],
    y=[x[0] for x in rmses_single_cnn_lstm],
    #color=[x[4] for x in rmses_single_cnn_lstm],
    title='RMSE - CNN LSTM Single Station'
    )

cnn_lstm_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

cnn_lstm_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_cnn_lstm]))

cnn_lstm_rmse_single.show()

#### Individual Station Training

In [39]:
rmses_indiv_cnn_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    cnn_lstm_indiv_model, cnn_lstm_indiv_rmse = cnn_lstm(station_data, features, target, n_steps, epochs)

    rmses_indiv_cnn_lstm.append((cnn_lstm_indiv_rmse, station, lat, lon))

rmses_indiv_cnn_lstm.sort(key = lambda x: x[0])

In [40]:
cnn_lstm_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_cnn_lstm],
    y=[x[0] for x in rmses_indiv_cnn_lstm],
    #color=[x[4] for x in rmses_single_multi_lstm],
    title='RMSE - CNN LSTM Indiv Station'
    )

cnn_lstm_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

cnn_lstm_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_cnn_lstm]))

multi_lstm_rmse_indiv.show()

### ConvLSTM

#### Single Station Training

In [41]:
n_steps = 28
epochs = 15

conv_lstm_single_model, conv_lstm_single_rmse = conv_lstm(single_station_train, features, target, n_steps, epochs)

rmses_single_conv_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)

    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    color = 'red' if station == station_selected else 'blue'

    predictions, rmse, fig = pred_conv_lstm(station_data, date_col, features, target, n_steps, conv_lstm_single_model, True)
    rmses_single_conv_lstm.append((rmse, station, lat, lon, color))

rmses_single_conv_lstm.sort(key = lambda x: x[0])

In [42]:
conv_lstm_rmse_single = px.bar( 
    x=[str(x[1]) for x in rmses_single_conv_lstm],
    y=[x[0] for x in rmses_single_conv_lstm],
    #color=[x[4] for x in rmses_single_cnn_lstm],
    title='RMSE - ConvLSTM Single Station'
    )

conv_lstm_rmse_single.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

conv_lstm_rmse_single.add_hline(y = np.mean([x[0] for x in rmses_single_conv_lstm]))

conv_lstm_rmse_single.show()

#### Individual Station Training

In [43]:
rmses_indiv_conv_lstm = []

for station in data['station'].unique():
    station_data = data.loc[data['station'] == station].reset_index(drop=True)
    lat = station_data['latitude'].values[0]
    lon = station_data['longitude'].values[0]

    conv_lstm_indiv_model, conv_lstm_indiv_rmse = conv_lstm(station_data, features, target, n_steps, epochs)

    rmses_indiv_conv_lstm.append((conv_lstm_indiv_rmse, station, lat, lon))

rmses_indiv_conv_lstm.sort(key = lambda x: x[0])

In [44]:
conv_lstm_rmse_indiv = px.bar( 
    x=[str(x[1]) for x in rmses_indiv_conv_lstm],
    y=[x[0] for x in rmses_indiv_conv_lstm],
    #color=[x[4] for x in rmses_single_cnn_lstm],
    title='RMSE - ConvLSTM Single Station'
    )

conv_lstm_rmse_indiv.update_layout(
        title_x=0.5,
        xaxis_title = 'Station',
        yaxis_title = 'RMSE',
        xaxis={'categoryorder':'total ascending'}
    )

conv_lstm_rmse_indiv.add_hline(y = np.mean([x[0] for x in rmses_indiv_conv_lstm]))

conv_lstm_rmse_indiv.show()

### RMSE Overview

In [45]:
rmse_df = pd.DataFrame()
rmse_df['single_lin_reg'] = sorted([x[0] for x in rmses_single_lin_reg])
rmse_df['multi_lin_reg'] = sorted([x[0] for x in rmses_multi_lin_reg])
rmse_df['indiv_lin_reg'] = sorted([x[0] for x in rmses_indiv_lin_reg])
rmse_df['single_bin_glm_reg'] = sorted([x[0] for x in rmses_single_bin_glm])
rmse_df['multi_bin_glm_reg'] = sorted([x[0] for x in rmses_multi_bin_glm])
rmse_df['indiv_bin_glm_reg'] = sorted([x[0] for x in rmses_indiv_bin_glm])
rmse_df['single_cnn_reg'] = sorted([x[0] for x in rmses_single_cnn])
rmse_df['multi_cnn_reg'] = sorted([x[0] for x in rmses_multi_cnn])
rmse_df['indiv_cnn_reg'] = sorted([x[0] for x in rmses_indiv_cnn])
rmse_df['single_multi_lstm_reg'] = sorted([x[0] for x in rmses_single_multi_lstm])
rmse_df['indiv_multi_lstm_reg'] = sorted([x[0] for x in rmses_indiv_multi_lstm])
rmse_df['single_cnn_lstm_reg'] = sorted([x[0] for x in rmses_single_cnn_lstm])
rmse_df['indiv_cnn_lstm_reg'] = sorted([x[0] for x in rmses_indiv_cnn_lstm])
rmse_df['single_conv_lstm_reg'] = sorted([x[0] for x in rmses_single_conv_lstm])
rmse_df['indiv_conv_lstm_reg'] = sorted([x[0] for x in rmses_indiv_conv_lstm])

rmse_fig = px.line(rmse_df)
rmse_fig.show()

In [46]:
from scipy import stats

p_value_df = pd.DataFrame(columns = rmse_df.columns.tolist(), index = rmse_df.columns.tolist())
t_value_df = pd.DataFrame(columns = rmse_df.columns.tolist(), index = rmse_df.columns.tolist())
alpha = 0.05

def independent_ttest(data1, data2, alpha):
    # calculate means
    mean1, mean2 = np.mean(data1), np.mean(data2)
    # calculate standard errors
    se1, se2 = stats.sem(data1), stats.sem(data2)
    # standard error on the difference between the samples
    sed = np.sqrt(se1**2.0 + se2**2.0)
    # calculate the t statistic
    t_stat = (mean1 - mean2) / sed
    # degrees of freedom
    df = len(data1) + len(data2) - 2
    # calculate the critical value
    cv = stats.t.ppf(1.0 - alpha, df)
    # calculate the p-value
    p = (1.0 - stats.t.cdf(abs(t_stat), df)) * 2.0 # Change to 1.0 for one tailed t test
    # return everything
    return t_stat, df, cv, p

for col in rmse_df.columns:
    for i in rmse_df.columns:
        #t_statistics = stats.ttest_ind(rmse_df[col], rmse_df[i])
        t_statistics = independent_ttest(rmse_df[col], rmse_df[i], alpha)
        t_value_df.loc[i, col] = t_statistics[0]
        p_value_df.loc[i, col] = t_statistics[3]

dof = t_statistics[1]
cv = t_statistics[2]

conclusion = pd.DataFrame(columns = rmse_df.columns.tolist() + ['sum'], index = rmse_df.columns.tolist())

for col in t_value_df.columns:
    for i in t_value_df.index:
        res = None
        if p_value_df.loc[i, col] < alpha:
            res = 1
        else:
            res = 0
        conclusion.loc[i, col] = res

for i in conclusion.index:
    conclusion.loc[i, 'sum'] = conclusion.loc[i].sum()


In [47]:
conclusion.sort_values('sum', ascending=False)['sum']

indiv_cnn_lstm_reg       13
indiv_multi_lstm_reg     11
indiv_conv_lstm_reg      10
indiv_cnn_reg             9
single_lin_reg            8
multi_lin_reg             8
indiv_lin_reg             8
single_conv_lstm_reg      8
single_cnn_lstm_reg       7
indiv_bin_glm_reg         6
single_multi_lstm_reg     6
single_bin_glm_reg        5
multi_bin_glm_reg         5
single_cnn_reg            5
multi_cnn_reg             5
Name: sum, dtype: object

In [55]:
conclusion.to_excel('one_tailed_t_test_results.xlsx')

In [48]:
t_stat_heatmap = px.imshow(conclusion, text_auto=True)
t_stat_heatmap.show()

In [49]:
p_value_heatmap = px.imshow(p_value_df, text_auto=True)
p_value_heatmap.show()

In [50]:
rmse_summary= px.bar( 
    y=rmse_df.describe().columns,
    x=rmse_df.describe().loc['mean'].values,
    title='Mean RMSE'
    )

rmse_summary.update_layout(
        title_x=0.5,
        xaxis_title = 'Mean RMSE',
        yaxis_title = 'Model',
        yaxis={'categoryorder':'total descending'}
    )

rmse_summary.show()

In [56]:
final_test_data = total_data.loc[total_data['station'] == station_selected].reset_index(drop=True)
final_test_data['precipitation'] = precip_encoder.transform(final_test_data['precipitation'])

final_model, final_rmse = cnn_lstm(final_test_data.iloc[:20000], features, target, n_steps, 50)

In [57]:
final_predictions, final_rmse_, final_fig = pred_cnn_lstm(final_test_data.iloc[20000:], date_col, features, target, n_steps, final_model, True)

In [58]:
final_fig.show()

In [54]:
lat_init = data.loc[data['station'] == station_selected]['latitude'].values[0]
lon_init = data.loc[data['station'] == station_selected]['longitude'].values[0]

sp = fo.Map(location=[lat_init, lon_init], zoom_start=8)

for i in rmses_indiv_cnn_lstm:
    fo.CircleMarker(
        [i[2], i[3]],
        radius=50*i[0],
        color='blue',
        fill_color='blue',
        fill=True,
        fill_opacity=0.7
        ).add_to(sp)
sp